In [ ]:
# Setting options for the plots
%matplotlib inline
%config InlineBackend.figure_formats={'retina', 'svg'}
%config InlineBackend.rc={'savefig.dpi': 150}

# Experiment Report 

In [ ]:
import itertools
import math
import os
import re
import pickle
import platform
import time
import warnings

from functools import partial
from os.path import abspath, relpath, exists, join

import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import statsmodels.api as sm
from matplotlib import pyplot as plt
from textwrap import wrap

# allow older versions of pandas to work
try:
    from pandas.io.common import DtypeWarning
except ImportError:
    from pandas.errors import DtypeWarning

from IPython import sys_info
from IPython.display import display, HTML, Image, Javascript, Markdown, SVG
from rsmtool.reader import DataReader
from rsmtool.writer import DataWriter
from rsmtool.utils.files import parse_json_with_comments
from rsmtool.utils.notebook import (float_format_func,
                                    int_or_float_format_func,
                                    compute_subgroup_plot_params,
                                    bold_highlighter,
                                    color_highlighter,
                                    show_thumbnail)

from rsmtool.fairness_utils import (get_fairness_analyses,
                                    write_fairness_results)

from rsmtool.version import VERSION as rsmtool_version

# turn off interactive plotting
plt.ioff()

sns.set_context('notebook')

In [ ]:
rsm_report_dir = os.environ.get('RSM_REPORT_DIR', None)
if rsm_report_dir is None:
    rsm_report_dir = os.getcwd()

rsm_environ_config = join(rsm_report_dir, '.environ.json')
if not exists(rsm_environ_config):
    raise FileNotFoundError('The file {} cannot be located. '
                            'Please make sure that either (1) '
                            'you have set the correct directory with the `RSM_REPORT_DIR` '
                            'environment variable, or (2) that your `.environ.json` '
                            'file is in the same directory as your notebook.'.format(rsm_environ_config))
    
environ_config = parse_json_with_comments(rsm_environ_config)

<style type="text/css">
  div.prompt.output_prompt { 
    color: white; 
  }
  
  span.highlight_color {
    color: red;
  }
  
  span.highlight_bold {
    font-weight: bold;  
  }
    
  @media print {
    @page {
      size: landscape;
      margin: 0cm 0cm 0cm 0cm;
    }

    * {
      margin: 0px;
      padding: 0px;
    }

    #toc {
      display: none;
    }

    span.highlight_color, span.highlight_bold {
        font-weight: bolder;
        text-decoration: underline;
    }

    div.prompt.output_prompt {
      display: none;
    }
    
    h3#Python-packages, div#packages {
      display: none;
  }
</style>

In [ ]:
# NOTE: you will need to set the following manually
# if you are using this notebook interactively.
experiment_id = environ_config.get('EXPERIMENT_ID')
description = environ_config.get('DESCRIPTION')
context = environ_config.get('CONTEXT')
train_file_location = environ_config.get('TRAIN_FILE_LOCATION')
test_file_location = environ_config.get('TEST_FILE_LOCATION')
output_dir = environ_config.get('OUTPUT_DIR')
figure_dir = environ_config.get('FIGURE_DIR')
model_name = environ_config.get('MODEL_NAME')
model_type = environ_config.get('MODEL_TYPE')
skll_fixed_parameters = environ_config.get('SKLL_FIXED_PARAMETERS')
skll_objective = environ_config.get('SKLL_OBJECTIVE')
file_format = environ_config.get('FILE_FORMAT')
length_column = environ_config.get('LENGTH_COLUMN')
second_human_score_column = environ_config.get('H2_COLUMN')
use_scaled_predictions = environ_config.get('SCALED')
min_score = environ_config.get("MIN_SCORE")
max_score = environ_config.get("MAX_SCORE")
standardize_features = environ_config.get('STANDARDIZE_FEATURES')
exclude_zero_scores = environ_config.get('EXCLUDE_ZEROS')
feature_subset_file = environ_config.get('FEATURE_SUBSET_FILE', ' ')
min_items = environ_config.get('MIN_ITEMS')
use_thumbnails = environ_config.get('USE_THUMBNAILS')
predict_expected_scores = environ_config.get('PREDICT_EXPECTED_SCORES')
rater_error_variance = environ_config.get("RATER_ERROR_VARIANCE")

# groups for analysis by prompt or subgroup.
groups_desc = environ_config.get('GROUPS_FOR_DESCRIPTIVES') 
groups_eval = environ_config.get('GROUPS_FOR_EVALUATIONS') 

# min number of n for group to be displayed in the report
min_n_per_group = environ_config.get('MIN_N_PER_GROUP')

if min_n_per_group is None:
    min_n_per_group = {}

# javascript path
javascript_path = environ_config.get("JAVASCRIPT_PATH")

In [ ]:
# initialize counter for thumbnail IDs
id_generator = itertools.count(1)

In [ ]:
with open(join(javascript_path, "sort.js"), "r", encoding="utf-8") as sortf:
    display(Javascript(data=sortf.read()))

In [ ]:
Markdown('''This report presents the analysis for **{}**: {}'''.format(experiment_id, description))

In [ ]:
markdown_str = ''
if use_thumbnails:
    markdown_str += ("""\n  - Images in this report have been converted to """
                     """clickable thumbnails.""")
if predict_expected_scores:
    markdown_str += ("""\n  - Predictions analyzed in this report are *expected scores*, """
                     """i.e., probability-weighted averages over all score points.""")

if markdown_str:
    markdown_str = '**Notes**:' + markdown_str
    display(Markdown(markdown_str))

In [ ]:
HTML(time.strftime('%c'))

In [ ]:
%%html
<div id="toc"></div>

In [ ]:
# Read in the training and testing features, both raw and pre-processed
# Make sure that the `spkitemid` and `candidate` columns are read as strings 
# to preserve any leading zeros
# We filter DtypeWarnings that pop up mostly in very large files

string_columns = ['spkitemid', 'candidate']
converter_dict = {column: str for column in string_columns}

with warnings.catch_warnings():
    warnings.filterwarnings('ignore', category=DtypeWarning)
    if exists(train_file_location):
        df_train_orig = DataReader.read_from_file(train_file_location)

    train_file = join(output_dir, '{}_train_features.{}'.format(experiment_id,
                                                                file_format))
    if exists(train_file):
        df_train = DataReader.read_from_file(train_file, converters=converter_dict)

    train_metadata_file = join(output_dir, '{}_train_metadata.{}'.format(experiment_id,
                                                                         file_format))    
    if exists(train_metadata_file):
        df_train_metadata = DataReader.read_from_file(train_metadata_file, converters=converter_dict)

    train_other_columns_file = join(output_dir, '{}_train_other_columns.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(train_other_columns_file):
        df_train_other_columns = DataReader.read_from_file(train_other_columns_file, converters=converter_dict)

    train_length_file = join(output_dir, '{}_train_response_lengths.{}'.format(experiment_id,
                                                                               file_format))
    if exists(train_length_file):
        df_train_length = DataReader.read_from_file(train_length_file, converters=converter_dict)

    train_excluded_file = join(output_dir, '{}_train_excluded_responses.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(train_excluded_file):
        df_train_excluded = DataReader.read_from_file(train_excluded_file, converters=converter_dict)

    train_responses_with_excluded_flags_file = join(output_dir, '{}_train_responses_with_excluded_flags.{}'.format(experiment_id,
                                                                                                                   file_format))
    if exists(train_responses_with_excluded_flags_file):
        df_train_responses_with_excluded_flags = DataReader.read_from_file(train_responses_with_excluded_flags_file,
                                                                           converters=converter_dict)

    train_preproc_file = join(output_dir, '{}_train_preprocessed_features.{}'.format(experiment_id,
                                                                                     file_format))    
    if exists(train_preproc_file):
        df_train_preproc = DataReader.read_from_file(train_preproc_file, converters=converter_dict)

    if exists(test_file_location):
        df_test_orig = DataReader.read_from_file(test_file_location)

    test_file = join(output_dir, '{}_test_features.{}'.format(experiment_id,
                                                              file_format))
    if exists(test_file):
        df_test = DataReader.read_from_file(test_file, converters=converter_dict)

    test_metadata_file = join(output_dir, '{}_test_metadata.{}'.format(experiment_id,
                                                                       file_format))    
    if exists(test_metadata_file):
        df_test_metadata = DataReader.read_from_file(test_metadata_file, converters=converter_dict)

    test_other_columns_file = join(output_dir, '{}_test_other_columns.{}'.format(experiment_id,
                                                                                 file_format))
    if exists(test_other_columns_file):
        df_test_other_columns = DataReader.read_from_file(test_other_columns_file, converters=converter_dict)

    test_human_scores_file = join(output_dir, '{}_test_human_scores.{}'.format(experiment_id,
                                                                               file_format))
    if exists(test_human_scores_file):
        df_test_human_scores = DataReader.read_from_file(test_human_scores_file, converters=converter_dict)

    test_excluded_file = join(output_dir, '{}_test_excluded_responses.{}'.format(experiment_id,
                                                                                 file_format))
    if exists(test_excluded_file):
        df_test_excluded = DataReader.read_from_file(test_excluded_file, converters=converter_dict)

    test_responses_with_excluded_flags_file = join(output_dir, '{}_test_responses_with_excluded_flags.{}'.format(experiment_id,
                                                                                                                 file_format))
    if exists(test_responses_with_excluded_flags_file):
        df_test_responses_with_excluded_flags = DataReader.read_from_file(test_responses_with_excluded_flags_file,
                                                                          converters=converter_dict)

    test_preproc_file = join(output_dir, '{}_test_preprocessed_features.{}'.format(experiment_id,
                                                                                   file_format))
    if exists(test_preproc_file):
        df_test_preproc = DataReader.read_from_file(test_preproc_file, converters=converter_dict)

    pred_preproc_file = join(output_dir, '{}_pred_processed.{}'.format(experiment_id,
                                                                       file_format))
    if exists(pred_preproc_file):
        df_pred_preproc = DataReader.read_from_file(pred_preproc_file, converters=converter_dict)

    feature_file = join(output_dir, '{}_feature.{}'.format(experiment_id,
                                                           file_format))
    if exists(feature_file):
        df_features = DataReader.read_from_file(feature_file, converters=converter_dict)
        features_used = [c for c in df_features.feature.values]
        # compute the longest feature name: we'll need if for the plots
        longest_feature_name = max(map(len, features_used))

    betas_file = join(output_dir, '{}_betas.{}'.format(experiment_id,
                                                       file_format))
    if exists(betas_file):
        df_betas = DataReader.read_from_file(betas_file)

    if exists(feature_subset_file):
        df_feature_subset_specs = DataReader.read_from_file(feature_subset_file)
    else:
        df_feature_subset_specs = None

In [ ]:
# check for continuous human scores in the evaluation set
continuous_human_score = False

if exists(pred_preproc_file):
    if not df_pred_preproc['sc1'].equals(np.round(df_pred_preproc['sc1'])):
        continuous_human_score = True

## Description of the data

In [ ]:
try:
    num_excluded_train = len(df_train_responses_with_excluded_flags)
except NameError:
    num_excluded_train = 0

try:
    num_excluded_test = len(df_test_responses_with_excluded_flags)
except NameError:
    num_excluded_test = 0

if context == 'rsmtool':
    pct_excluded_train = round(100*num_excluded_train/len(df_train_orig), 2)
pct_excluded_test = round(100*num_excluded_test/len(df_test_orig), 2)

if (num_excluded_train != 0 or num_excluded_test != 0):
    display(Markdown("### Responses excluded due to flags"))

    display(Markdown("Total number of responses excluded due to flags:"))
    if context=='rsmtool':
        display(Markdown("Training set: {} responses ({:.1f}% of the original {} responses)".format(num_excluded_train, pct_excluded_train, len(df_train_orig))))
    display(Markdown("Evaluation set: {} responses ({:.1f}% of the original {} responses)".format(num_excluded_test, pct_excluded_test, len(df_test_orig))))


### Responses excluded due to non-numeric feature values or scores

In [ ]:
try:
    num_missing_rows_train = len(df_train_excluded)
except NameError:
    num_missing_rows_train = 0

if context == 'rsmtool':
    pct_missing_rows_train = 100*num_missing_rows_train/len(df_train_orig)

try:
    num_missing_rows_test = len(df_test_excluded)
except:
    num_missing_rows_test = 0
pct_missing_rows_test = 100*num_missing_rows_test/len(df_test_orig)

In [ ]:
excluded_candidates_note = Markdown("Note: if a candidate had less than {} responses left for analysis after applying all filters, "
                                    "all responses from that "
                                    "candidate were excluded from further analysis.".format(min_items))

In [ ]:
if context == 'rsmtool':
    display(Markdown("**Training set**"))
    display(Markdown('Total number of excluded responses: {} ({:.1f}% of the original {})'.format(num_missing_rows_train, pct_missing_rows_train, len(df_train_orig))))
    if num_missing_rows_train != 0:
        train_excluded_analysis_file = join(output_dir, '{}_train_excluded_composition.{}'.format(experiment_id,
                                                                                                  file_format))
        df_train_excluded_analysis = DataReader.read_from_file(train_excluded_analysis_file)
        display(HTML(df_train_excluded_analysis.to_html(classes=['sortable'], float_format=float_format_func, index=False))) 
        if min_items > 0:
            display(excluded_candidates_note)

In [ ]:
display(Markdown('**Evaluation set**'))
display(Markdown('Total number of excluded responses: {} ({:.1f}% of the original {})'.format(num_missing_rows_test, pct_missing_rows_test, len(df_test_orig))))
if num_missing_rows_test != 0:
    test_excluded_analysis_file = join(output_dir, '{}_test_excluded_composition.{}'.format(experiment_id,
                                                                                            file_format))
    df_test_excluded_analysis = DataReader.read_from_file(test_excluded_analysis_file)
    display(HTML(df_test_excluded_analysis.to_html(classes=['sortable'], float_format=float_format_func, index=False)))
    if min_items > 0:
        display(excluded_candidates_note)

The rest of this report is based only on the responses that were not excluded above.

In [ ]:
if context == 'rsmtool':
    display(Markdown('### Composition of the training and evaluation sets'))
elif context == 'rsmeval':
    display(Markdown('### Composition of the evaluation set'))

In [ ]:
# show the table showing candidate (speaker), prompt 
# and responses stats for training and test

# feature descriptives extra table
data_composition_file = join(output_dir, '{}_data_composition.{}'.format(experiment_id, file_format))
df_data_desc = DataReader.read_from_file(data_composition_file)
display(HTML(df_data_desc.to_html(classes=['sortable'], float_format=float_format_func, index=False)))

try:
    num_double_scored_responses = len(df_test_human_scores[df_test_human_scores['sc2'].notnull()])
except NameError:
    pass
else:
    zeros_included_or_excluded = 'excluded' if exclude_zero_scores else 'included'
    display(Markdown("Total number of double scored responses in the evaluation set" 
                     " used: {} (zeros {})".format(num_double_scored_responses,
                                                   zeros_included_or_excluded)))

In [ ]:
# This notebook displays data composition tables for all groups specified in groups_desc variable.

In [ ]:
for group in groups_desc:
    display(Markdown('### Number of responses by {}'.format(group)))
    data_composition_by_file = join(output_dir, '{}_data_composition_by_{}.{}'.format(experiment_id,
                                                                                      group,
                                                                                      file_format))
    df_data_composition_by_group = DataReader.read_from_file(data_composition_by_file)
    display(HTML(df_data_composition_by_group.to_html(classes=['sortable'], float_format=float_format_func, index=False)))
    if group in min_n_per_group:
        display(Markdown("Note: the rest of this report will only display the results for groups with "
                         "at least {} responses in the partition used for each set of analyses.".format(min_n_per_group[group])))

## Overall descriptive feature statistics

These values are reported before transformations.

In [ ]:
# feature descriptives table
desc_file = join(output_dir, '{}_feature_descriptives.{}'.format(experiment_id, file_format))

df_desc = DataReader.read_from_file(desc_file, index_col=0)
HTML(df_desc.to_html(classes=['sortable'], float_format=float_format_func))

### Prevalence of recoded cases

This sections shows the number and percentage of cases truncated to mean +/- 4 SD for each feature.

In [ ]:
outliers_file = join(output_dir, '{}_feature_outliers.{}'.format(experiment_id, file_format))
df_outliers = DataReader.read_from_file(outliers_file, index_col=0)
df_outliers.index.name = 'feature'
df_outliers = df_outliers.reset_index()
df_outliers = pd.melt(df_outliers, id_vars=['feature'])
df_outliers = df_outliers[df_outliers.variable.str.contains(r'[ulb].*?perc')]


# we need to increase the plot height if feature names are long
if longest_feature_name > 10:
    height = 3 + math.ceil((longest_feature_name - 10)/10)
else:
    height = 3
    
# we also need a higher aspect if we have more than 40 features
# The aspect defines the final width of the plot (width=aspect*height).
# We keep the width constant (9 for plots with many features or 6
# for plots with few features) by dividing the expected width
# by the height. 
aspect = 9/height if len(features_used) > 40 else 6/height


# colors for the plot
colors = sns.color_palette("Greys", 3)

# what's the largest value in the data frame
maxperc = df_outliers['value'].max()

# compute the limits for the graph
limits = (0, max(2.5, maxperc))

with sns.axes_style('whitegrid'):
    # create a barplot without a legend since we will manually
    # add one later
    p = sns.catplot(x="feature", y="value", hue="variable", kind="bar", 
                    palette=colors, data=df_outliers, height=height, 
                    aspect=aspect, legend=False)
    p.set_axis_labels('', '% cases truncated\nto mean +/- 4*sd')
    p.set_xticklabels(rotation=90)
    p.set(ylim=limits)

    # add a line at 2%
    axis = p.axes[0][0]
    axis.axhline(y=2.0, linestyle='--', linewidth=1.5, color='black')

    # add a legend with the right colors
    legend=axis.legend(('both', 'lower', 'upper'), title='', frameon=True, fancybox=True, ncol=3)
    legend.legendHandles[0].set_color(colors[0])
    legend.legendHandles[1].set_color(colors[1])

    # we want to try to force `tight_layout()`, but if this 
    # raises a warning, we don't want the entire notebook to fail
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        plt.tight_layout(h_pad=1.0)

    imgfile = join(figure_dir, '{}_outliers.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

### Feature value distribution

The following table shows additional statistics for the data. Quantiles are computed using type=3 method used in SAS. The mild outliers are defined as data points between [1.5, 3) \* IQR away from the nearest quartile. Extreme outliers are the data points >= 3 * IQR away from the nearest quartile.

In [ ]:
# feature descriptives extra table
desce_file = join(output_dir, '{}_feature_descriptivesExtra.{}'.format(experiment_id,
                                                                       file_format))
df_desce = DataReader.read_from_file(desce_file, index_col=0)
HTML(df_desce.to_html(classes=['sortable'], float_format=float_format_func))

In [ ]:
# This notebook generates boxplot with feature values for all groups specified in groups_desc variable. 
# Note that this analysis uses the raw feature values after filtering out the missing features and scores.

# merge metadata and feature values
df_train_merged = pd.merge(df_train, df_train_metadata, on = 'spkitemid')

In [ ]:
num_features = len(features_used)

if (num_features > 150):
    display(Markdown('### Feature values by subgroup(s)'))
    display(Markdown('Since the data has {} (> 150) features, boxplots with feature values for all groups '
                     'will be skipped.'.format(num_features)))
elif (30 < num_features <= 150 and not use_thumbnails):
    display(Markdown('### Feature values by subgroup(s)'))
    display(Markdown('Since the data has {} (> 30 but <= 150) features, you need to set `"use_thumbnails"` to `true` in your '
                     'configuration file to generate boxplots with feature values for all groups.'.format(num_features)))
else:
    for group in groups_desc:
        display(Markdown('### Feature values by {}'.format(group)))
        display(Markdown('In all plots in this subsection the values are reported before '
                         'transformations/truncation. The lines indicate the threshold for '
                         'truncation (mean +/- 4*SD).'))

        df_train_feats = df_train_merged[features_used + [group]]
        
        # if we have threshold set for this group, filter the data now
        if group in min_n_per_group:
            display(Markdown("The report only shows the results for groups with "
                             "at least {} responses in the training set.".format(min_n_per_group[group])))

            category_counts = df_train_merged[group].value_counts()
            selected_categories = category_counts[category_counts >= min_n_per_group[group]].index

            df_train_feats_all = df_train_merged[df_train_merged[group].isin(selected_categories)].copy()
        else:
        
            df_train_feats_all = df_train_merged.copy()
        
        if len(df_train_feats_all) > 0:
        
            df_train_feats_all[group] = 'All data'

            df_train_combined = pd.concat([df_train_feats, df_train_feats_all], sort=True)
            df_train_combined.reset_index(drop=True, inplace=True)

            # Define the order of the boxes: put 'All data' first and 'No info' last.
            group_levels = sorted(list(df_train_feats[group].unique()))
            if 'No info' in group_levels:
                box_names = ['All data'] + [level for level in group_levels if level != 'No info'] + ['No info']
            else:
                box_names = ['All data'] + group_levels

            # create the faceted boxplots
            fig = plt.figure()
            (figure_width, 
             figure_height, 
             num_rows, 
             num_columns, 
             wrapped_box_names) = compute_subgroup_plot_params(box_names, num_features)

            fig.set_size_inches(figure_width, figure_height)
            with sns.axes_style('white'), sns.plotting_context('notebook', font_scale=1.2):
                for i, varname in enumerate(sorted(features_used)):
                    df_plot = df_train_combined[[group, varname]]
                    min_value = df_plot[varname].mean() - 4 * df_plot[varname].std()
                    max_value = df_plot[varname].mean() + 4 * df_plot[varname].std()
                    ax = fig.add_subplot(num_rows, num_columns, i + 1)
                    ax.axhline(y=float(min_value), linestyle='--', linewidth=0.5, color='r')
                    ax.axhline(y=float(max_value), linestyle='--', linewidth=0.5, color='r')
                    sns.boxplot(x=df_plot[group], y=df_plot[varname], color='#b3b3b3', ax=ax, order=box_names)
                    ax.set_xticklabels(wrapped_box_names, rotation=90) 
                    ax.set_xlabel('')
                    ax.set_ylabel('')
                    plot_title = '{} by {}'.format('\n'.join(wrap(varname, 30)), group)
                    ax.set_title(plot_title)

            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                plt.tight_layout(h_pad=1.0)


            # save the figure as an SVG file.
            imgfile = join(figure_dir, '{}_feature_boxplot_by_{}.svg'.format(experiment_id, group))
            plt.savefig(imgfile)
            if use_thumbnails:
                show_thumbnail(imgfile, next(id_generator))
            else:
                # needed so that the figures are shown after the heading and not at the end of the cell
                plt.show()
        else:
            display(Markdown("None of the groups in {} had {} or more responses.".format(group,
                                                                                         min_n_per_group[group])))

##  Feature distributions and inter-feature correlations

### Training set distributions

The following plot shows the distributions of the feature values in 
the training set, after transformation (if applicable), truncation 
and standardization (if applicable). The line shows the kernel density estimate. The 
human score (`sc1`) is also included. 

Response length (`length`) is included if you specified `length_column` in the config file, unless
the column had missing values or a standard deviation <= 0.

In [ ]:
selected_columns = features_used + ['sc1', 'spkitemid']
df_train_preproc_selected_features = df_train_preproc[selected_columns]
try:
    df_train_preproc_selected_features = df_train_preproc_selected_features.merge(df_train_length, on='spkitemid')
except NameError:
    column_order = sorted(features_used) + ['sc1']
else:
    column_order = sorted(features_used) + ['sc1', 'length']

df_train_preproc_melted = pd.melt(df_train_preproc_selected_features, id_vars=['spkitemid'])
df_train_preproc_melted = df_train_preproc_melted[['variable', 'value']]

# we need to reduce col_wrap and increase width if the feature names are too long
if longest_feature_name > 10:
    col_wrap = 2
    # adjust height to allow for wrapping really long names. We allow 0.25 in per line
    height = 2+(math.ceil(longest_feature_name/30)*0.25)
    aspect = 4/height
else:
    col_wrap = 3
    aspect = 1


with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    with sns.axes_style('white'):
        g = sns.FacetGrid(col='variable', data=df_train_preproc_melted, col_wrap=col_wrap, 
                          col_order=column_order, sharex=False, sharey=False, height=height, 
                          aspect=aspect)
        g.map(sns.distplot, "value", color="grey", kde=False)
        for ax, cname in zip(g.axes, g.col_names):
            labels = ax.get_xticks()
            ax.set_xlabel('')
            ax.set_xticklabels(labels, rotation=90)
            plot_title = '\n'.join(wrap(str(cname), 30))
            ax.set_title(plot_title)

        plt.tight_layout(h_pad=1.0)
        imgfile = join(figure_dir, '{}_distrib.svg'.format(experiment_id))
        plt.savefig(imgfile)
        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()

### Inter-feature correlations

The following table shows the Pearson correlations between all the training features
after transformation (if applicable), truncation and standardization (if applicable). The human score 
(`sc1`) is also included. 

Response length (`length`) is included if 
you specified `length_column` in the config file, unless the column had missing 
values or a standard deviation <= 0. 

The following values are <span class="highlight_color">highlighted</span>:
- inter-feature correlations above 0.7, and
- `sc1`-feature correlations lower than 0.1 or higher than 0.7

In [ ]:
cors_file = join(output_dir, '{}_cors_processed.{}'.format(experiment_id,
                                                           file_format))
df_cors = DataReader.read_from_file(cors_file, index_col=0)
if 'length' in df_cors.columns:
    feature_columns = sorted([c for c in df_cors.columns if c not in ['sc1', 'length']])
    order = ['sc1', 'length'] + feature_columns
else:
    feature_columns = sorted([c for c in df_cors.columns if c != 'sc1'])
    order = ['sc1'] + feature_columns
    
df_cors = df_cors.reindex(index=order, columns=order)

# wrap the column names if the feature names are very long
if longest_feature_name > 10:
    column_names = ['\n'.join(wrap(c, 10)) for c in order]
else:
    column_names = order
    
df_cors.columns = column_names

# apply two different formatting to the columns according
# to two different thresholds. The first one highlights all
# inter-feature correlations > 0.7 (so, not including sc1)
# and the second highlights all sc1-X correlations lower
# than 0.1 and higher than 0.7. We will use red for the
# first formatting and blue for the second one. 
formatter1 = partial(color_highlighter, low=-1, high=0.7)
formatter2 = partial(color_highlighter, low=0.1, high=0.7)

formatter_dict = {c: formatter1 for c in column_names if not c == 'sc1'}
formatter_dict.update({'sc1': formatter2})

HTML(df_cors.to_html(classes=['sortable'], formatters=formatter_dict, escape=False))

### Marginal and partial correlations

The plot below shows correlations between truncated and standardized (if applicable) values of each feature against human score. The first bar (`Marginal`) in each case shows Pearson's correlation. The second bar (`Partial - all`) shows partial correlations after controlling for all other variables. If you specified `length_column` in the config file, a third bar (`Partial - length`) will show partial correlations of each feature against the human score after controlling for length. The dotted lines correspond to *r* = 0.1 and *r* = 0.7.

In [ ]:
# read in and merge the score correlations 
margcor_file = join(output_dir, '{}_margcor_score_all_data.{}'.format(experiment_id, file_format))
pcor_file = join(output_dir, '{}_pcor_score_all_data.{}'.format(experiment_id, file_format))

df_margcor = DataReader.read_from_file(margcor_file, index_col=0)
df_pcor = DataReader.read_from_file(pcor_file, index_col=0)

# check if we have length partial correlations
pcor_no_length_file = join(output_dir, '{}_pcor_score_no_length_all_data.{}'.format(experiment_id,
                                                                                    file_format))
with_length = exists(pcor_no_length_file)
if with_length:
    df_pcor_no_length = DataReader.read_from_file(pcor_no_length_file, index_col=0)
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], 
                             df_pcor.loc['All data'], 
                             df_pcor_no_length.loc['All data']]).transpose()
    df_mpcor.columns = ['marginal', 'partial_all', 'partial_length']
    num_entries = 3
    labels = ('Marginal', 'Partial - all', 'Partial - length')

else:
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], 
                             df_pcor.loc['All data']]).transpose()
    df_mpcor.columns = ['marginal', 'partial_all']
    num_entries = 2
    labels = ('Marginal', 'Partial (all)')

df_mpcor.index.name = 'feature'
df_mpcor = df_mpcor.reset_index()
df_mpcor = pd.melt(df_mpcor, id_vars=['feature'])

# we need to change the plot height if the feature names are long
if longest_feature_name > 10:
    height = 3 + math.ceil((longest_feature_name - 10)/10)
else:
    height = 3
        
# we need a higher aspect if we have more than 40 features
aspect = 9/height if len(features_used) > 40 else 6/height


# get the colors for the plot
colors = sns.color_palette("Greys", num_entries)

# check for any negative correlations
limits = (0, 1)
if len(df_mpcor[df_mpcor.value < 0]):
    limits = (-1, 1)


with sns.axes_style('whitegrid'):

    # generate a bar plot but without the legend since we will
    # manually add one later
    p = sns.catplot(x="feature", y="value", hue="variable", kind="bar",
                    palette=colors, data=df_mpcor, height=height, 
                    aspect=aspect, legend=False)
    p.set_axis_labels('', 'Correlation with score')
    p.set_xticklabels(rotation=90)
    p.set(ylim=limits)
    
    # add a line at 0.1 and 0.7
    axis = p.axes[0][0]
    axis.axhline(y=0.1, linestyle='--', linewidth=0.5, color='black');
    axis.axhline(y=0.7, linestyle='--', linewidth=0.5, color='black');

    # create the legend manually with the right colors
    legend = axis.legend(labels=labels, title='', frameon=True, 
                         fancybox=True, ncol=num_entries)
    for i in range(num_entries):
        legend.legendHandles[i].set_color(colors[i]);

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        plt.tight_layout(h_pad=1.0)

    imgfile = join(figure_dir, '{}_cors_score.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

In [ ]:
len_margcor_file = join(output_dir, '{}_margcor_length_all_data.{}'.format(experiment_id,
                                                                           file_format))
len_pcor_file = join(output_dir, '{}_pcor_length_all_data.{}'.format(experiment_id,
                                                                     file_format))
if exists(len_margcor_file) and exists(len_pcor_file):

    if standardize_features:
        display(Markdown("The plot below shows the same correlations between truncated and "
                         "standardized values of each feature against length."))
    else:
        display(Markdown("The plot below shows the same correlations between truncated and "
                         "un-standardized values of each feature against length."))

    df_margcor = DataReader.read_from_file(len_margcor_file, index_col=0)
    df_pcor = DataReader.read_from_file(len_pcor_file, index_col=0)
    df_mpcor = pd.DataFrame([df_margcor.loc['All data'], df_pcor.loc['All data']]).transpose()
    df_mpcor.index.name = 'feature'
    df_mpcor.columns = ['marginal', 'partial']
    df_mpcor = df_mpcor.reset_index()
    df_mpcor = pd.melt(df_mpcor, id_vars=['feature'])

    # we need to change the plot height if the feature names are long
    if longest_feature_name > 10:
        height = 3 + math.ceil((longest_feature_name - 10)/10)
    else:
        height = 3
        
    # we need a higher aspect if we have more than 40 features
    aspect = 9/height if len(features_used) > 40 else 6/height


    # check for any negative correlations
    limits = (0, 1)
    if len(df_mpcor[df_mpcor.value < 0]):
        limits = (-1, 1)

    # get the colors for the plot
    colors = sns.color_palette("Greys", 2)
        
    with sns.axes_style('whitegrid'):
        
        # create a barplot but without the legend since
        # we will manually add one later
        p = sns.catplot(x="feature", y="value", hue="variable", kind="bar",
                        palette=colors, data=df_mpcor, height=height, 
                        aspect=aspect, legend=False)
        p.set_axis_labels('', 'Correlation with length')
        p.set_xticklabels(rotation=90)
        p.set(ylim=limits)

        # create the legend manually with the right colors
        axis = p.axes[0][0]
        legend = axis.legend(labels=('Marginal', 'Partial  - all'), title='', 
                             frameon=True, fancybox=True, ncol=2)
        legend.legendHandles[0].set_color(colors[0]);
        legend.legendHandles[1].set_color(colors[1]);
        imgfile = join(figure_dir, '{}_cors_length.svg'.format(experiment_id))
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            plt.tight_layout(h_pad=1.0)

        plt.savefig(imgfile) 
        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()

In [ ]:
if len(groups_desc) > 0:
    markdown_str = ["## Differential feature functioning"]
    markdown_str.append("This section shows differential feature functioning (DFF) plots "
                        "for all features and subgroups. The features are shown after applying "
                        "transformations (if applicable) and truncation of outliers.")
    display(Markdown('\n'.join(markdown_str)))

In [ ]:
# check if we already created the merged file in another notebook

try:
    df_train_preproc_merged
except NameError:
    df_train_preproc_merged = pd.merge(df_train_preproc, df_train_metadata, on = 'spkitemid')

for group in groups_desc:
    display(Markdown("### DFF by {}".format(group)))
    
    if group in min_n_per_group:
        display(Markdown("The report only shows the results for groups with "
                         "at least {} responses in the training set.".format(min_n_per_group[group])))
        
        category_counts = df_train_preproc_merged[group].value_counts()
        selected_categories = category_counts[category_counts >= min_n_per_group[group]].index
        
        df_train_preproc_selected = df_train_preproc_merged[df_train_preproc_merged[group].isin(selected_categories)].copy()
    else:
        df_train_preproc_selected = df_train_preproc_merged.copy()
    
    
    if len(df_train_preproc_selected) > 0:
        
        # we need to reduce col_wrap and increase width if the feature names are too long
        if longest_feature_name > 10:
            col_wrap = 2
            # adjust height to allow for wrapping really long names. We allow 0.25 in per line
            height = 2+(math.ceil(longest_feature_name/30)*0.25)
            aspect = 5/height
            # show legend near the second plot in the grid
            plot_with_legend = 1
        else:
            height=3
            col_wrap = 3
            aspect = 1
            # show the legend near the third plot in the grid
            plot_with_legend = 2
        
        selected_columns = ['spkitemid', 'sc1'] + features_used + [group]
        df_melted = pd.melt(df_train_preproc_selected[selected_columns], id_vars=['spkitemid', 'sc1', group], var_name='feature')
        group_values = sorted(df_melted[group].unique())
        colors = sns.color_palette("Greys", len(group_values))
        with sns.axes_style('whitegrid'), sns.plotting_context('notebook', font_scale=1.2):
            p = sns.catplot(x='sc1', y='value', hue=group, hue_order = group_values,
                            col='feature', col_wrap=col_wrap, height=height, aspect=aspect,
                            scale=0.6,
                            palette=colors,
                            sharey=False, sharex=False, legend=False, kind="point",
                            data=df_melted)

            for i, axis in enumerate(p.axes):
                axis.set_xlabel('score')
                if i == plot_with_legend:
                    legend = axis.legend(group_values, title=group, 
                                         frameon=True, fancybox=True, 
                                         ncol=1, fontsize=10,
                                         loc='upper right', bbox_to_anchor=(1.75, 1))
                    for j in range(len(group_values)):
                        legend.legendHandles[j].set_color(colors[j])
                    plt.setp(legend.get_title(), fontsize='x-small')
            
            for ax, cname in zip(p.axes, p.col_names):
                ax.set_title('\n'.join(wrap(str(cname), 30)))


            with warnings.catch_warnings():
                warnings.simplefilter('ignore')
                plt.tight_layout(h_pad=1.0)

            imgfile = join(figure_dir, '{}_dff_{}.svg'.format(experiment_id, group))
            plt.savefig(imgfile)
            if use_thumbnails:
                show_thumbnail(imgfile, next(id_generator))
            else:
                plt.show()
    else:
        display(Markdown("None of the groups in {} had {} or more responses.".format(group,
                                                                                    min_n_per_group[group])))

In [ ]:
markdown_strs = ['## Consistency']

consistency_file = join(output_dir, '{}_consistency.{}'.format(experiment_id, file_format))
degradation_file = join(output_dir, '{}_degradation.{}'.format(experiment_id, file_format))
disattenuation_file = join(output_dir, '{}_disattenuated_correlations.{}'.format(experiment_id, file_format))
eval_file = join(output_dir, '{}_eval.{}'.format(experiment_id,
                                                 file_format))

if exists(consistency_file) and exists(degradation_file) and exists(disattenuation_file):
    df_consistency = DataReader.read_from_file(consistency_file, index_col=0)
    df_degradation = DataReader.read_from_file(degradation_file, index_col=0)
    df_dis_corrs = DataReader.read_from_file(disattenuation_file, index_col=0)
    df_eval = DataReader.read_from_file(eval_file, index_col=0)

    markdown_strs.append('*Note: this section assumes that the score used for evaluating machine scores '
                         'is the score assigned by the first rater.*')
    markdown_strs.append('### Human-human agreement')
    markdown_strs.append("This table shows the human-human agreement on the "
                         "double-scored evaluation data.")
    if continuous_human_score:
        markdown_strs.append('For the computation of `kappa` and `wtkappa` '
                             'human scores have beeen rounded to the nearest integer.')
        
    markdown_strs.append("The following are <span class='highlight_color'>highlighted </span>: ")
    markdown_strs.append(' - Exact agreement (`exact_agr`) < 50%')
    markdown_strs.append(' - Adjacent agreement (`adj_agr`) < 95%')
    markdown_strs.append(' - Quadratic weighted kappa (`wtkappa`) < 0.7')
    markdown_strs.append(' - Pearson correlation (`corr`) < 0.7')
    display(Markdown('\n'.join(markdown_strs)))
    
    # display the HTML for the table with the various formatters
    formatter_exact_agr = partial(color_highlighter, low=50, high=100)
    formatter_adj_agr = partial(color_highlighter, low=95, high=100)
    formatter_wtkappa_corr = partial(color_highlighter, low=0.7)
    formatter_dict = {'exact_agr': formatter_exact_agr, 
                      'adj_agr': formatter_adj_agr,
                      'wtkappa': formatter_wtkappa_corr, 
                      'corr': formatter_wtkappa_corr}
    display(HTML(df_consistency.to_html(index=False,
                                        escape=False,
                                        float_format=float_format_func,
                                        formatters=formatter_dict)))
    
    markdown_strs = ['### Degradation']
    markdown_strs.append('The next table shows the degradation in the evaluation metrics '
                         '(`diff`) when comparing the machine (`H-M`) to a second human (`H-H`). '
                         'A positive degradation value indicates better human-machine performance. '
                         'Note that the human-machine agreement is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'agreement is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following degradation values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `corr` < -0.1')
    markdown_strs.append(' - `wtkappa` < -0.1')
    display(Markdown('\n'.join(markdown_strs)))
    df_eval_for_degradation = df_eval[df_degradation.columns].copy()
    df_consistency_for_degradation = pd.concat([df_consistency]*len(df_eval), sort=True)
    df_consistency_for_degradation = df_consistency_for_degradation[df_degradation.columns].copy()
    df_consistency_for_degradation.index = df_eval_for_degradation.index

    df_consistency_for_degradation['type'] = 'H-H'
    df_eval_for_degradation['type'] = 'H-M'
    df_degradation['type'] = 'diff'

    df = pd.concat([df_consistency_for_degradation,
                    df_eval_for_degradation,
                    df_degradation], sort=True)
    df = df[['type','corr', 'kappa', 'wtkappa', 'exact_agr', 'adj_agr', 'SMD']]
    df = df.reset_index()
    df = df.set_index(['index', 'type']).sort_index(level='index')
    df.index.names = [None, None]
    
    # display the HTML for the table with the various formatters
    formatter_corr = partial(color_highlighter, low=-0.1, high=100)
    formatter_wtkappa = partial(color_highlighter, low=-0.1, high=100)
    formatter_dict = {'corr': formatter_corr, 'wtkappa': formatter_wtkappa}
    display(HTML(df.to_html(float_format=float_format_func, 
                            formatters=formatter_dict, escape=False)))
    
    
    markdown_strs = ['### Disattenuated correlations']
    markdown_strs.append('The next table shows the correlations between human and machine scores, '
                         'the correlations between two human scores, '  
                         'and disattenuated correlations between human and machine scores computed as '
                         'human-machine correlations divided by the square root of human-human '
                         'correlation. '
                         'Note that the human-machine correlation is computed on the full '
                         'dataset (to get a reliable estimate) whereas the human-human '
                         'correlation is computed on the subset of responses that were double-scored.')
    markdown_strs.append("\nThe following values are "
                         "<span class='highlight_color'>highlighted</span>")
    markdown_strs.append(' - `disattenuated_corr` < -0.9')
    display(Markdown('\n'.join(markdown_strs)))
    # display the HTML for the table with the various formatters
    formatter_dis_corr = partial(color_highlighter, low=0.9)
    formatter_dict = {'corr_disattenuated': formatter_dis_corr}
    display(HTML(df_dis_corrs.to_html(index=True,
                                      escape=False,
                                      classes=['sortable'],
                                      float_format=float_format_func,
                                      formatters=formatter_dict)))
else:  
    markdown_strs.append("The configuration file did not specify "
                         "`second_human_score_column` which is necessary to compute "
                         "consistency metrics.")
    display(Markdown('\n'.join(markdown_strs)))
    
    

## Model

In [ ]:
Markdown('Model used: **{}**'.format(model_name))

In [ ]:
Markdown('Number of features in model: **{}**'.format(len(features_used)))

In [ ]:
builtin_ols_models = ['LinearRegression',
                      'EqualWeightsLR',
                      'RebalancedLR',
                      'NNLR',
                      'LassoFixedLambdaThenNNLR',
                      'LassoFixedLambdaThenLR',
                      'PositiveLassoCVThenLR',
                      'WeightedLeastSquares']

builtin_lasso_models = ['LassoFixedLambda',
                        'PositiveLassoCV']

In [ ]:
# we first just show a summary of the OLS model and the main model parameters
if model_name in builtin_ols_models:
    display(Markdown('### Model summary'))
    summary_file = join(output_dir, '{}_ols_summary.txt'.format(experiment_id))
    with open(summary_file, 'r') as summf:
        model_summary = summf.read()
        print(model_summary)
     
    display(Markdown('### Model fit'))
    df_fit = DataReader.read_from_file(join(output_dir, '{}_model_fit.{}'.format(experiment_id,
                                                                                 file_format)))
    display(HTML(df_fit.to_html(index=False,
                                float_format=float_format_func)))

### Standardized and relative regression coefficients (betas)

The relative coefficients are intended to show relative contribution of different feature and their primary purpose is to indentify whether one of the features has an unproportionate effect over the final score. They are computed as standardized/(sum of absolute values of standardized coefficients). 

Negative standardized coefficients are <span class="highlight_color">highlighted</span>.

**Note**: if the model contains negative coefficients, relative values will not sum up to one and their interpretation is generally questionable. 

In [ ]:
markdown_str = """
**Note**: The coefficients were estimated using LASSO regression. Unlike OLS (standard) linear regression, lasso estimation is based on an optimization routine and therefore the exact estimates may differ across different systems. """

if model_name in builtin_lasso_models:
    display(Markdown(markdown_str))

In [ ]:
df_betas.sort_values(by='feature', inplace=True)
display(HTML(df_betas.to_html(classes=['sortable'], 
                              index=False, 
                              escape=False,
                              float_format=float_format_func,
                              formatters={'standardized': color_highlighter})))

Here are the same values, shown graphically.

In [ ]:
df_betas_sorted = df_betas.sort_values(by='standardized', ascending=False)
df_betas_sorted.reset_index(drop=True, inplace=True)
fig = plt.figure()
fig.set_size_inches(8, 3)
fig.subplots_adjust(bottom=0.5)
grey_colors = sns.color_palette('Greys', len(features_used))[::-1]
with sns.axes_style('whitegrid'):
    ax1=fig.add_subplot(121)
    sns.barplot(x="feature", y="standardized", data=df_betas_sorted, 
                order=df_betas_sorted['feature'].values,
                palette=sns.color_palette("Greys", 1), ax=ax1)
    ax1.set_xticklabels(df_betas_sorted['feature'].values, rotation=90)
    ax1.set_title('Values of standardized coefficients')
    ax1.set_xlabel('')
    ax1.set_ylabel('')
    # no pie chart if we have more than 15 features,
    # if the feature names are long (pie chart looks ugly)
    # or if there are any negative coefficients.
    if len(features_used) <= 15 and longest_feature_name <= 10 and (df_betas_sorted['relative']>=0).all():
        ax2=fig.add_subplot(133, aspect=True)
        ax2.pie(abs(df_betas_sorted['relative'].values), colors=grey_colors, 
            labels=df_betas_sorted['feature'].values, normalize=True)
        ax2.set_title('Proportional contribution of each feature')
    else:
        fig.set_size_inches(len(features_used), 3)
    betas_file = join(figure_dir, '{}_betas.svg'.format(experiment_id))
    plt.savefig(betas_file)

    if use_thumbnails:
        show_thumbnail(betas_file, next(id_generator))
    else:
        plt.show()

In [ ]:
if model_name in builtin_ols_models:
    display(Markdown('### Model diagnostics'))
    display(Markdown("These are standard plots for model diagnostics for the main model. All information is computed based on the training set."))

In [ ]:
# read in the OLS model file and create the diagnostics plots
if model_name in builtin_ols_models:
    ols_file = join(output_dir, '{}.ols'.format(experiment_id))
    model = pickle.load(open(ols_file, 'rb'))
    model_predictions = model.predict()

    with sns.axes_style('white'):
        f, (ax1, ax2) = plt.subplots(1, 2)
        f.set_size_inches((10, 4))
        
        ###
        # for now, we do not show the influence plot since it can be slow to generate
        ###
        # sm.graphics.influence_plot(model.sm_ols, criterion="cooks", size=10, ax=ax1)
        # ax1.set_title('Residuals vs. Leverage', fontsize=16)
        # ax1.set_xlabel('Leverage', fontsize=16)
        # ax1.set_ylabel('Standardized Residuals', fontsize=16)

        sm.qqplot(model.resid, stats.norm, fit=True, line='q', ax=ax1)
        ax1.set_title('Normal Q-Q Plot', fontsize=16)
        ax1.set_xlabel('Theoretical Quantiles', fontsize=16)
        ax1.set_ylabel('Sample Quantiles', fontsize=16)

        ax2.scatter(model_predictions, model.resid)
        ax2.set_xlabel('Fitted values', fontsize=16)
        ax2.set_ylabel('Residuals', fontsize=16)
        ax2.set_title('Residuals vs. Fitted', fontsize=16)

        imgfile = join(figure_dir, '{}_ols_diagnostic_plots.png'.format(experiment_id))
        plt.savefig(imgfile)

        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            display(Image(imgfile))
        plt.close()

## Evaluation results

### Overall association statistics

In [ ]:
markdown_str = ("The tables in this section show the standard association metrics between "
                "*observed* human scores and different types of machine scores. "
                "These results are computed on the evaluation set. `raw_trim` scores "
                "are truncated to [{}, {}]. `raw_trim_round` scores are computed by first truncating "
                "and then rounding the predicted score. Scaled scores are computed by re-scaling "
                "the predicted scores using mean and standard deviation of human scores as observed "
                "on the training data and mean and standard deviation of machine scores as predicted "
                "for the training set.".format(min_score, max_score))
display(Markdown(markdown_str))

#### Descriptive holistic score statistics

The table shows distributional properties of human and system scores. SMD values lower then -0.15 or higher than 0.15 are <span class="highlight_color">highlighted</span>.

*Please note that for raw scores, SMD values are likely to be affected by possible differences in scale.*

In [ ]:
raw_or_scaled = "scaled" if use_scaled_predictions else "raw"
eval_file = join(output_dir, '{}_eval.{}'.format(experiment_id, file_format))
df_eval = DataReader.read_from_file(eval_file, index_col=0)
distribution_columns = ['N', 'h_mean', 'sys_mean', 'h_sd',  'sys_sd', 'h_min', 'sys_min', 'h_max', 'sys_max', 'SMD']
association_columns = ['N'] + [column for column in df_eval.columns if not column in distribution_columns]
df_distribution = df_eval[distribution_columns]
df_association = df_eval[association_columns]

In [ ]:
pd.options.display.width=10
formatter = partial(color_highlighter, low=-0.15, high=0.15)
HTML('<span style="font-size:95%">'+ df_distribution.to_html(classes=['sortable'], 
                                                             escape=False,
                                                             formatters={'SMD': formatter},
                                                             float_format=float_format_func) + '</span>')

#### Association statistics


In [ ]:
markdown_str = ['The table shows the standard association metrics between human scores and machine scores.']
if continuous_human_score:
    markdown_str.append("Note that for computation of `kappa` both human and machine scores are rounded.")
else:
    markdown_str.append("Note that for computation of `kappa` all machine scores are rounded.")

Markdown('\n'.join(markdown_str))

In [ ]:
pd.options.display.width=10
HTML('<span style="font-size:95%">'+ df_association.to_html(classes=['sortable'], 
                                                            escape=False,
                                                            float_format=float_format_func) + '</span>')

### Confusion matrix

In [ ]:
markdown_str = ["Confusion matrix using {}, trimmed, and rounded scores and human scores (rows=system, columns=human).".format(raw_or_scaled)]

if continuous_human_score:
    markdown_str.append("Note: Human scores have beeen rounded to the nearest integer.")
            
Markdown('\n'.join(markdown_str))

In [ ]:
confmat_file = join(output_dir, '{}_confMatrix.{}'.format(experiment_id, file_format))
df_confmat = DataReader.read_from_file(confmat_file, index_col=0)
df_confmat

### Distribution of human and machine scores

In [ ]:
markdown_strs = ["The histogram and the table below show the distibution of "
                 "human scores and {}, trimmed, and rounded machine scores "
                 "(as % of all responses).".format(raw_or_scaled)]
markdown_strs.append("Differences in the table between human and machine distributions "
                     "larger than 5 percentage points are <span class='highlight_color'>highlighted</span>.")
if continuous_human_score:
    markdown_strs.append("Note: Human scores have beeen rounded to the nearest integer.")
    
display(Markdown('\n'.join(markdown_strs)))

In [ ]:
scoredist_file = join(output_dir, '{}_score_dist.{}'.format(experiment_id, file_format))
df_scoredist = DataReader.read_from_file(scoredist_file, index_col=0)
df_scoredist_melted = pd.melt(df_scoredist, id_vars=['score'])
df_scoredist_melted = df_scoredist_melted[df_scoredist_melted['variable'] != 'difference']

# get the colors for the plot
colors = sns.color_palette("Greys", 2)

with sns.axes_style('whitegrid'):

    # make a barplot without a legend since we will 
    # add one manually later
    p = sns.catplot(x="score", y="value", hue="variable", kind="bar",
                    palette=colors, data=df_scoredist_melted, 
                    height=3, aspect=2, legend=False)
    p.set_axis_labels('score', '% of responses')
    
    # add a legend with the right colors
    axis = p.axes[0][0]
    legend = axis.legend(labels=('Human', 'Machine'), title='', frameon=True, fancybox=True)
    legend.legendHandles[0].set_color(colors[0])
    legend.legendHandles[1].set_color(colors[1])

    imgfile = join(figure_dir, '{}_score_dist.svg'.format(experiment_id))
    plt.savefig(imgfile)

    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

In [ ]:
formatter = partial(color_highlighter, low=0, high=5, absolute=True)
df_html = df_scoredist.to_html(classes=['sortable'], index=False, 
                               escape=False, formatters={'difference': formatter})
display(HTML(df_html))

In [ ]:
markdown_strs = ['### True score evaluations']

raw_or_scaled = "scaled" if use_scaled_predictions else "raw"
true_eval_file = join(output_dir, '{}_true_score_eval.{}'.format(experiment_id, file_format))
if exists(true_eval_file): 
    df_true_eval = DataReader.read_from_file(true_eval_file, index_col=0)
    df_true_eval.replace({np.nan: '-'}, inplace=True)
    prmse_columns = ['N','N raters', 'N single', 'N multiple', 
                     'Variance of errors', 'True score var',
                     'MSE true', 'PRMSE true']
    df_prmse = df_true_eval[prmse_columns]

    markdown_strs.append("The tables in this section show how well system scores can "
                         "predict *true* scores. According to Test theory, a *true* score "
                         "is a score that would have been obtained if there were no errors "
                         "in measurement. While true scores cannot be observed, the variance "
                         "of true scores and the prediction error can be estimated using observed "
                         "human scores when multiple human ratings are available for a subset of "
                         "responses.")

    if rater_error_variance is None: 
        
        rater_variance_source = 'estimated'
        # if we estimated rater error variance from the data,
        # we display the variance of the two human raters
        # so that the user can verify there is no bias
        # We get that data from existing analyses
        df_human_variance = df_consistency[['N', 'h1_sd', 'h2_sd']].copy()
        df_human_variance['N_double'] = df_human_variance['N']
        df_human_variance['h1_var (double)'] = df_human_variance['h1_sd']**2
        df_human_variance['h2_var (double)'] = df_human_variance['h2_sd']**2
        df_human_variance['N_total'] = df_eval.iloc[0]['N']
        df_human_variance['h1_var (single)'] = df_eval.iloc[0]['h_sd']**2
        df_human_variance.index = ['human']


        if context == 'rsmtool':
            label_column = "test_label_column"
        else:
            label_column = "human_score_column"

        markdown_strs.append("In this notebook the variance of true scores is estimated using "
                             "the human ratings available for "
                             "responses in the evaluation set. Note that the analyses in this "
                             "section assume that the values "
                            "in `{}` and `second_human_score_column` are independent scores "
                            "from different raters or groups of raters. These analyses are "
                            "not applicable to a situation where `{}` contains an average "
                            "score from multiple raters".format(label_column, label_column))

        markdown_strs.append("#### Variance of human scores")
        markdown_strs.append("The table below shows variance of both sets of human scores "
                            "for the whole evaluation set and for the subset of responses "
                            "that were double-scored. Large differences in variance between "
                            "the two human scores require further investigation.")
        display(Markdown('\n'.join(markdown_strs)))
        pd.options.display.width=10
        column_order = ['N_total', 'N_double', 'h1_var (single)', 'h1_var (double)', 'h2_var (double)']
        display(HTML('<span style="font-size:95%">'+ df_human_variance[column_order].to_html(classes=['sortable'], 
                                                                                            escape=False,
                                                                                            float_format=float_format_func) + '</span>'))
    else:
        markdown_strs.append("In this notebook the variance of true scores was "
                            "estimated using the value of rater error variance "
                            "supplied by the user ({})".format(rater_error_variance))
        display(Markdown('\n'.join(markdown_strs)))
        rater_variance_source = 'supplied'
    
    
    markdown_strs = ["#### Proportional reduction in mean squared error (PRMSE)"]
    markdown_strs.append("The table shows {} variance of human rater errors, "
                         "true score variance, mean squared error (MSE) and "
                         "proportional reduction in mean squared error (PRMSE) for "
                         "predicting a true score with system score. As for other evaluations, "
                         "these results are computed on the evaluation set. `raw_trim` scores "
                         "are truncated to [{}, {}]. `raw_trim_round` scores are computed "
                         "by first truncating and then rounding the predicted score. Scaled scores "
                         "are computed by re-scaling the predicted scores using mean and standard "
                         "deviation of human scores as observed on the training data and mean and "
                         "standard deviation of machine scores as predicted for the training set.".format(rater_variance_source,
                                                                                                          min_score,
                                                                                                          max_score))
    display(Markdown('\n'.join(markdown_strs)))
    pd.options.display.width=10
    display(HTML('<span style="font-size:95%">'+ df_prmse.to_html(classes=['sortable'], 
                                                               escape=False,
                                                               float_format=float_format_func) + '</span>'))
else:
    markdown_strs.append("The configuration file did not specify "
                         "`second_human_score_column` or `rater_error_variance`. "
                         "At least one of these must be specified to compute "
                         "evaluations against true scores.")
    display(Markdown('\n'.join(markdown_strs)))

In [ ]:
# This notebook generates barplot with evaluation metrics for all groups specified in groups_eval variable. 


In [ ]:
basic_metrics = {('wtkappa', 'trim'): [0.7],
                 ('corr', 'trim'): [0.7],
                 ('DSM', 'trim_round'): [0.1, -0.1],
                 ('DSM', 'trim'): [0.1, -0.1],
                 ('R2', 'trim'): [],
                 ('RMSE', 'trim'): []}

colprefix = 'scale' if use_scaled_predictions else 'raw'
metrics = dict([('{}.{}_{}'.format(k[0], colprefix, k[1]), v) for k,v in basic_metrics.items()])
num_metrics = len(metrics)

for group in groups_eval:
    display(Markdown('### Evaluation by {}'.format(group)))
    
    eval_group_file = join(output_dir, '{}_eval_by_{}.{}'.format(experiment_id, group, file_format))
    df_eval_group_all = DataReader.read_from_file(eval_group_file, index_col=0)
    
    df_eval_group_all.index.name = group
    df_eval_group_all.reset_index(inplace=True)
    
    # If we have threshold per group, apply it now. Keep "All data" in any case. 
    if group in min_n_per_group:
        display(Markdown("The report only shows the results for groups with "
                         "at least {} responses in the evaluation set.".format(min_n_per_group[group])))
        
        df_eval_group = df_eval_group_all[(df_eval_group_all['N'] >= min_n_per_group[group]) |
                                         (df_eval_group_all[group] == 'All data')].copy()
    else:
        df_eval_group = df_eval_group_all.copy()


    # Define the order of the bars: put 'All data' first and 'No info' last.
    group_levels = list(df_eval_group[group])
    group_levels = [level for level in group_levels if level != 'All data']
    
    # We only want to show the report if we have anything other than All data
    if len(group_levels) > 0:
        
        if 'No info' in group_levels:
            bar_names = ['All data'] + [level for level in group_levels if level != 'No info'] + ['No info']
        else:
            bar_names = ['All data'] + group_levels

        fig = plt.figure()
        (figure_width, 
         figure_height, 
         num_rows, 
         num_columns, 
         wrapped_bar_names) = compute_subgroup_plot_params(bar_names, num_metrics)

        fig.set_size_inches(figure_width, figure_height)
        with sns.axes_style('white'), sns.plotting_context('notebook', font_scale=1.2):
            for i, metric in enumerate(sorted(metrics.keys())):
                df_plot = df_eval_group[[group, metric]]
                ax = fig.add_subplot(num_rows, num_columns, i + 1)
                for lineval in metrics[metric]:
                    ax.axhline(y=float(lineval), linestyle='--', linewidth=0.5, color='black')
                sns.barplot(x=df_plot[group], y=df_plot[metric], color='grey', ax=ax, order=bar_names)
                ax.set_xticklabels(wrapped_bar_names, rotation=90) 
                ax.set_xlabel('')
                ax.set_ylabel('')

                # set the y-limits of the plots appropriately
                if metric.startswith('corr') or metric.startswith('wtkappa'):
                    if df_plot[metric].min() < 0:
                        y_limits = (-1.0, 1.0)
                        ax.axhline(y=0.0, linestyle='--', linewidth=0.5, color='black')
                    else:
                        y_limits = (0.0, 1.0)
                    ax.set_ylim(y_limits)
                elif metric.startswith('R2'):
                    min_value = df_plot[metric].min()
                    if min_value < 0:
                        y_limits = (min_value - 0.1, 1.0)
                        ax.axhline(y=0.0, linestyle='--', linewidth=0.5, color='black')
                    else:
                        y_limits = (0.0, 1.0)
                    ax.set_ylim(y_limits)
                elif metric.startswith('RMSE'):
                    max_value = df_plot[metric].max()
                    y_limits = (0.0, max(max_value + 0.1, 1.0))
                    ax.set_ylim(y_limits)
                elif metric.startswith('DSM'):
                    min_value = df_plot[metric].min()
                    if min_value < 0:
                        ax.axhline(y=0.0, linestyle='--', linewidth=0.5, color='black')

                # set the title
                ax.set_title('{} by {}'.format(metric, group))

        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            plt.tight_layout(h_pad=1.0)

        imgfile = join(figure_dir, '{}_eval_by_{}.svg'.format(experiment_id, group))
        plt.savefig(imgfile)

        if use_thumbnails:
            show_thumbnail(imgfile, next(id_generator))
        else:
            plt.show()
            
    else:
        display(Markdown("None of the groups in {} had {} or more responses.".format(group,
                                                                                    min_n_per_group[group])))

## Additional fairness analyses

In [ ]:
# show a note for ETS users
from rsmtool import HAS_RSMEXTRA
if HAS_RSMEXTRA:
    from rsmextra.settings import fairness_note
    display(Markdown(fairness_note))

This section presents additional fairness analyses described in detail in [Loukina et al. (2019)](https://aclweb.org/anthology/papers/W/W19/W19-4401/).
These analyses consider separately different definitions of fairness and can assist in further trouble-shooting 
the observed subgroup differences. The evaluations focuses on three dimensions:

* **Outcome fairness** measures:  

    - *Overall score accuracy*: whether the automated scores are equally accurate for each group. The metric shows how much of the variance in squared error $(S-H)^2$ is explained by subgroup membership.

    -  *Overall score difference*: whether the automated scores are consistently different from human scores for members of a certain group. The metric shows how much of the variance in actual error $S-H$ is explained by subgroup membership. 

The differences in the outcome fairness measures might be due to different mean scores (different score distributions) across subgroups or due to differential treatment of different subgroups by the scoring engine or both. 

* **Process fairness** measures:

    - *Conditional score difference*: whether the automated scoring model assigns different scores to members from different groups despite them having the same construct proficiency. The metric shows how much additional variance in actual error ($S-H$) is explained by subgroup membership after controlling for human score, which can be thought of as a reasonable proxy for proficiency. 

The differences in process fairness measures indicate differential treatment of different subgroups by the scoring model.

In [7]:
# define and auxiliary function that we will need 
# for the plot later on
def errplot(x, y, xerr, **kwargs):
    ax = plt.gca()
    data = kwargs.pop("data")
    # let's remove color from kwargs
    color = kwargs.pop('color')
    data.plot(x=x, y=y, xerr=xerr,
              kind="barh", ax=ax,
              color=colors,
              **kwargs)


In [ ]:
# check if we already created the merged file in another notebook

try:
    df_pred_preproc_merged
except NameError:
    df_pred_preproc_merged = pd.merge(df_pred_preproc, df_test_metadata, on = 'spkitemid')
    
# check which score we are using
system_score_column = "scale_trim" if use_scaled_predictions else "raw_trim"

for group in groups_eval:
    
    display(Markdown("### Additional fairness evaluation by {}".format(group)))
    
    # run the actual analyses. We are currently doing this in the notebook
    # so that it is easy to skip these if necessary. The notebook
    # and analyses are set up in a way that will make it easy to move these
    # in future to the main pipeline and read in the outputs here. 
    fit_dict, fairness_container = get_fairness_analyses(df_pred_preproc_merged,
                                                         group,
                                                         system_score_column)
    
    # write the results to disk so that we can consider them in tests
    write_fairness_results(fit_dict,
                           fairness_container,
                           group,
                           output_dir,
                           experiment_id,
                           file_format)
    
    # first show summary results
    df_fairness = fairness_container['fairness_metrics_by_{}'.format(group)]

    
    display(Markdown("The summary table shows the overall score accuracy (OSA), overall score difference (OSD) "
                    "and conditional score difference (CSD). The first row reports the percentage of variance, "
                    " explained by group membership, the second row shows $p$ value. "
                     "{} was used as a reference category. "
                    "Larger values of R2 indicate larger differences between subgroups. " 
                    "Further detail about each model can be found in [intermediate "
                    "output files](#Links-to-Intermediate-Files).".format(df_fairness['base_category'].values[0])))
    
    display(HTML(df_fairness.loc[['R2', 'sig'],
                                 ['Overall score accuracy',
                                  'Overall score difference',
                                  'Conditional score difference']].to_html(classes='sortable',
                                                                     float_format=float_format_func)))

    
    Markdown_str = [("The plots show error estimates for different categories for each group "
                    "(squared error for OSA, raw error for OSD, and conditional raw error for CSD) "
                    "The estimates have been adjusted for the value of the group used as the Intercept. "
                    "Black lines show 95% confidence intervals estimated by the model.")]
    
    # if we only care about groups above threshold, identify those.
    
    category_counts = df_pred_preproc_merged[group].value_counts()        
    
    if group in min_n_per_group:
        Markdown_str.append("While the models were fit to all data, the plots only show estimates for "
                            "categories with more than {} members and the Intercept ({}).".format(min_n_per_group[group],
                                                                                                  df_fairness['base_category'].values[0]))
        
        groups_by_size = category_counts[category_counts >= min_n_per_group[group]].index
        df_pred_preproc_selected = df_pred_preproc_merged[df_pred_preproc_merged[group].isin(groups_by_size)].copy()
    else:
        groups_by_size = category_counts.index
        df_pred_preproc_selected = df_pred_preproc_merged.copy()
    
    display(Markdown('\n'.join(Markdown_str)))
   
    
    if len(groups_by_size) > 0:

        # assemble all coefficients into a long data frame
        all_coefs = []
        for metrics in ['osa', 'csd', 'osd']:
            df_metrics = fairness_container['estimates_{}_by_{}'.format(metrics, group)].copy()
            # compute adjusted error estimates by adding the value of the Intercept
            # to all non-Intercept values
            non_index_cols = [r for r in df_metrics.index if not "Intercept" in r]
            index_col = [r for r in df_metrics.index if "Intercept" in r]
            df_metrics['error_estimate'] = df_metrics['estimate']
            df_metrics.loc[non_index_cols,
                          'error_estimate'] = df_metrics.loc[non_index_cols,
                                                            'estimate'] + df_metrics.loc[index_col,
                                                                                        'estimate'].values
            # create a column for metrics
            df_metrics['metrics'] = metrics
            # only use groups with values above threshold and the intercept
            df_metrics[group] = df_metrics.index
            df_metrics_selected = df_metrics[df_metrics[group].isin(groups_by_size) |
                                             (df_metrics[group] == index_col[0])]
            all_coefs.append(df_metrics_selected)


        # show coefficient plots
        # define groups and color palette
        colors = sns.color_palette("Greys_r", len(groups_by_size))

        df_coefs_all = pd.concat(all_coefs)

        # compute the size of the confidence interval from the boundary
        df_coefs_all['CI'] = np.abs(df_coefs_all['[0.025'] - df_coefs_all['estimate'])

        # plot the coefficients
        with sns.axes_style('whitegrid'), sns.plotting_context('notebook', font_scale=2):
            g = sns.FacetGrid(df_coefs_all, col="metrics",
                              height=10, col_order = ['osa', 'osd', 'csd'])
            g.map_dataframe(errplot, group, "error_estimate",  "CI").set_axis_labels("Error estimate",
                                                                                    group)

            imgfile = join(figure_dir, '{}_fairness_estimates_{}.svg'.format(experiment_id, group))
            plt.savefig(imgfile)
            if use_thumbnails:
                show_thumbnail(imgfile, next(id_generator))
            else:
                plt.show()


        # Show the conditional score plot
        markdown_str = [("The plot shows average {} system score for each "
                        "group conditioned "
                         "on human score.".format(system_score_column))]
        if group in min_n_per_group:
            markdown_str.append("The plot only shows estimates for "
                               "categories with more than {} members and the"
                                "reference category ({}).".format(min_n_per_group[group],
                                                          df_fairness['base_category'].values[0]))
        
        display(Markdown('\n'.join(markdown_str)))
        


        with sns.axes_style('whitegrid'), sns.plotting_context('notebook', font_scale=1.2):
            p = sns.catplot(x='sc1', y=system_score_column,
                            hue=group, 
                            hue_order = groups_by_size,
                            palette=colors,
                            legend_out=True,
                            kind="point",
                            data=df_pred_preproc_selected)

            #plt.tight_layout(h_pad=1.0)
            imgfile = join(figure_dir, '{}_conditional_score_{}.svg'.format(experiment_id, group))
            plt.savefig(imgfile)
            if use_thumbnails:
                show_thumbnail(imgfile, next(id_generator))
            else:
                plt.show()
    else:
        display(Markdown("None of the groups in {} had {} or more responses in the evaluation set.".format(group,
                                                                                     min_n_per_group[group])))
            

## Principal component analysis

PCA using scaled data and singular value decomposition. This is computed using processed features after the truncation of outliers and other transformations specified in feature config file.

In [ ]:
pca_file = join(output_dir, '{}_pca.{}'.format(experiment_id, file_format))
df_pca = DataReader.read_from_file(pca_file, index_col=0)
df_pca.sort_index(inplace=True)
HTML(df_pca.to_html(classes=['sortable'], float_format=float_format_func))

In [ ]:
pcavar_file = join(output_dir, '{}_pcavar.{}'.format(experiment_id, file_format))
df_pcavar = DataReader.read_from_file(pcavar_file, index_col=0)
df_pcavar.sort_index(inplace=True)
HTML(df_pcavar.to_html(classes=['sortable'], float_format=float_format_func))

In [ ]:
# generate the Scree plot
with sns.axes_style('white'):
    num_components = len(df_pcavar.columns)
    labels = list(df_pcavar.columns)
    ax = df_pcavar.transpose().plot(y='Eigenvalues', kind='line', 
                                    color='black', linestyle='dashed', marker='o', legend=False,
                                    linewidth=1, use_index=True, xticks=range(num_components),
                                    figsize=(11, 5), title='Scree Plot: Principal Component Analysis')
    ax.set_ylabel('Variances')
    ax.set_xticklabels(labels, rotation=90)
    imgfile = join(figure_dir, '{}_pca.svg'.format(experiment_id))
    plt.savefig(imgfile)
    if use_thumbnails:
        show_thumbnail(imgfile, next(id_generator))
    else:
        plt.show()

## Links to intermediate files

Click on the hyperlinks below to see the intermediate experiment files generated as part of this experiment.

In [ ]:
from rsmtool.utils.notebook import show_files

In [ ]:
show_files(output_dir, experiment_id, file_format)

## System information

In [ ]:
system_name = platform.system()

# People might not know what 'Darwin' is, so we should replace that with 'Mac OS X'
if system_name == 'Darwin':
    system_name = 'Mac OS X'
    
# get the architecture
architecture = platform.architecture()[0]

# get the rsmtool version
rsmtool_version_str = '.'.join(map(str, rsmtool_version))

display(Markdown('This report was generated using rsmtool v{} on a '
                 '{} computer running {}.'.format(rsmtool_version_str, 
                                                  architecture, 
                                                  system_name)))

### Python packages

In [ ]:
import pkg_resources
package_names = '\n'.join(sorted(["%s==%s" % (i.key, i.version) for i in pkg_resources.working_set]))
display(HTML('<div id="packages"><pre>{}</pre></div>'.format(package_names)))

In [ ]:
%%javascript

// Code to dynamically generate table of contents at the top of the HTML file
var tocEntries = ['<ul>'];
var anchors = $('a.anchor-link');
var headingTypes = $(anchors).parent().map(function() { return $(this).prop('tagName')});
var headingTexts = $(anchors).parent().map(function() { return $(this).text()});
var subList = false;

$.each(anchors, function(i, anch) {
    var hType = headingTypes[i];
    var hText = headingTexts[i];
    hText = hText.substr(0, hText.length - 1);
    if (hType == 'H2') {
        if (subList) {
            tocEntries.push('</ul>')
            subList = false;
        }
        tocEntries.push('<li><a href="' + anch + '"</a>' + hText + '</li>')
    }
    else if (hType == 'H3') {
        if (!subList) {
            subList = true;
            tocEntries.push('<ul>')
        }
        tocEntries.push('<li><a href="' + anch + '"</a>' + hText + '</li>')
    }
});
tocEntries.push('</ul>')
$('#toc').html(tocEntries.join(' '))